In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori ,fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import re

原始資料讀取和檢視

Input: user_review_Model_Name_with_category.csv

In [2]:
modelname = 'MOBIUZ'

In [4]:
raw = pd.read_csv('user_review_v1_'+modelname+'.csv') #根據model名修改檔名
raw = raw.drop(columns=['Unnamed: 0'])
#raw = raw.drop(columns='review_date_palce')
for i in range(len(raw)):
    raw['Asin'][i] = re.sub(r'[^a-zA-Z0-9]',"",raw['Asin'][i])
raw.index = range(len(raw.index))
raw

,user_name,product_name,review_date_place,review_rating,review_content,Asin
0,Brandon Lopez,MSR90 USB Swipe Magnetic Credit Card Reader 3 ...,"reviewed a product · Mar 21, 2021",5 out of five stars,It’s simple just plug it in and it’s ready to ...,B01DUB4GVO
1,Brandon Lopez,NADAMOO Floor and Furniture Repair Kit 18 Colo...,"reviewed a product · Mar 21, 2021",5 out of five stars,I was able to fill in a 3/4 hole I drilled int...,B07ZR5DSMQ
2,Brandon Lopez,SHOP LC DELIVERING JOY Home Table Decor Desk F...,"reviewed a product · Mar 21, 2021",3 out of five stars,"As the title says there are no switches, butto...",B08DP4SVC7
3,Brandon Lopez,Wireless Mouse Noise-Free 2.4G Full Size Recha...,"reviewed a product · Mar 21, 2021",5 out of five stars,Nice RGB not to expensive I would say this mou...,B07ZQM54HP
4,Brandon Lopez,CABLEDECONN 3M 10FT DisplayPort 1.4 Cable 8K@6...,"reviewed a product · Mar 21, 2021",5 out of five stars,Good quality able to output 4K @ 120fps,B08L7T8RSP
...,...,...,...,...,...,...
4821,West Indies,BenQ EL2870U 28 inch 4K Monitor for Gaming 1ms...,"reviewed a product · Sep 13, 2020",5 out of five stars,I bought this for my 4K Security Camera System...,B078HWN5CX
4822,West Indies,Universal Security Camera Base Mount Junction ...,"reviewed a product · Sep 8, 2020",5 out of five stars,Solid Built. I must say I am really impressed ...,B07NQVV385
4823,West Indies,"Gentle Jewelry Cleaner Solution | Gold, Silver...","reviewed a product · May 3, 2017",1 out of five stars,I followed the instructions on the product lab...,B0182UTI5W
4824,West Indies,Bose Companion 2 Series III Multimedia Speaker...,"reviewed a product · Aug 10, 2016",3 out of five stars,These speakers don't sound great. There is not...,B00CD1PTF0


Merge Category with Asin

Input: ModelNameAsin_Category_list.csv

In [5]:
Category_list = pd.read_csv(f'Asin_Category_{modelname}_list.csv')
Category_list = Category_list.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
for i in range(len(Category_list)):
    for cl in ['Category','Sub_category']:
        Category_list[cl][i] = re.sub(r'[^a-zA-Z0-9]'," ",Category_list[cl][i])
    for cla in ['Asin']:
        Category_list[cla][i] = re.sub(r'[^a-zA-Z0-9]',"",Category_list[cla][i])

Category_list

,Asin,Brand,Category,Sub_category,Breakdown_category,Price
0,B01DUB4GVO,['None'],OfficeElectronics,CreditCardReaders,StationaryCreditCardReaders,['17.80']
1,B07ZR5DSMQ,['None'],HouseholdSupplies,WoodPolishCare,ScratchCoversRemovers,['Non-Available']
2,B08DP4SVC7,['None'],LightingCeilingFans,LampsShades,DeskLamps,['39.99']
3,B07ZQM54HP,[' Geyes '],ComputersAccessories,Keyboards MiceAccessories,Mice,['21.99']
4,B08L7T8RSP,[' Deconn '],ComputersAccessories,ComputerCableAdapters,DVI-HDMIAdapters,['11.99']
...,...,...,...,...,...,...
3287,B077VXV323,[' SanDisk '],ComputersAccessories,DataStorage,USBFlashDrives,['7.53']
3288,B07W3QM1YR,['None'],NintendoSwitch,Faceplates ProtectorsSkins,ScreenProtectors,['Non-Available']
3289,B078VM9RVB,['None'],NoveltyMore,HatsCaps,BaseballCaps,['16.98']
3290,B0841JFFCR,[' HK GAMING '],ComputersAccessories,Keyboards MiceAccessories,Keyboards,['42.99']


In [6]:
#print(Category_list['Asin'][0])
#print(raw['Asin'][0])
format = pd.merge(raw,Category_list)
clean_format = format.drop_duplicates()
clean_format.index = range(len(clean_format.index))
clean_format

,user_name,product_name,review_date_place,review_rating,review_content,Asin,Brand,Category,Sub_category,Breakdown_category,Price
0,Brandon Lopez,MSR90 USB Swipe Magnetic Credit Card Reader 3 ...,"reviewed a product · Mar 21, 2021",5 out of five stars,It’s simple just plug it in and it’s ready to ...,B01DUB4GVO,['None'],OfficeElectronics,CreditCardReaders,StationaryCreditCardReaders,['17.80']
1,Brandon Lopez,NADAMOO Floor and Furniture Repair Kit 18 Colo...,"reviewed a product · Mar 21, 2021",5 out of five stars,I was able to fill in a 3/4 hole I drilled int...,B07ZR5DSMQ,['None'],HouseholdSupplies,WoodPolishCare,ScratchCoversRemovers,['Non-Available']
2,Brandon Lopez,SHOP LC DELIVERING JOY Home Table Decor Desk F...,"reviewed a product · Mar 21, 2021",3 out of five stars,"As the title says there are no switches, butto...",B08DP4SVC7,['None'],LightingCeilingFans,LampsShades,DeskLamps,['39.99']
3,Brandon Lopez,Wireless Mouse Noise-Free 2.4G Full Size Recha...,"reviewed a product · Mar 21, 2021",5 out of five stars,Nice RGB not to expensive I would say this mou...,B07ZQM54HP,[' Geyes '],ComputersAccessories,Keyboards MiceAccessories,Mice,['21.99']
4,Brandon Lopez,CABLEDECONN 3M 10FT DisplayPort 1.4 Cable 8K@6...,"reviewed a product · Mar 21, 2021",5 out of five stars,Good quality able to output 4K @ 120fps,B08L7T8RSP,[' Deconn '],ComputersAccessories,ComputerCableAdapters,DVI-HDMIAdapters,['11.99']
...,...,...,...,...,...,...,...,...,...,...,...
3552,ITATI,"Hocase for Galaxy S21 Plus Case, (with 2 Scree...","reviewed a product · Mar 13, 2021",5 out of five stars,10/10!,B08S2YLJSC,[' Hocase '],Cases HolstersSleeves,Cases HolstersSleeves,BasicCases,['11.99']
3553,ITATI,BRIGHTWORLD Moon Lamp Kids Night Light Galaxy ...,"reviewed a product · Mar 13, 2021",5 out of five stars,Amazing Moon Lamp 10/10!!!,B07PY7GLKV,['None'],LightingCeilingFans,WallLights,NightLights,['20.39']
3554,James,Kailh Box Switches for Mechanical Gaming Keybo...,"reviewed a product · Mar 8, 2021",5 out of five stars,Love the sturdiness and thock these have. My f...,B0841JFFCR,[' HK GAMING '],ComputersAccessories,Keyboards MiceAccessories,Keyboards,['42.99']
3555,West Indies,"Amazon Basics High-Speed HDMI Cable (18Gbps, 4...","reviewed a product · Sep 13, 2020",5 out of five stars,Great Quality Cable. I Will definitely recomme...,B075ZWHNJT,[' Amazon Basics '],AccessoriesSupplies,VideoCables,HDMICables,['17.54']


In [7]:
#原始Category數量
clean_format.value_counts('Category')

Category
  ComputersAccessories                     1098
  Men                                       130
  KitchenDining                             123
  Accessories                               100
  AccessoriesSupplies                        97
                                           ... 
  History                                     1
  SmallAppliancePartsAccessories              1
  HeavyDutyCommercialVehicleEquipment         1
  Health FitnessDieting                       1
  AEHomeVideo                                 1
Length: 224, dtype: int64

資料前處理: 1.根據商品名重新分類 2.時間格式處理

In [8]:
re_cate_format = clean_format


In [9]:
#調整時間格式
print('時間處理前:',type(re_cate_format['review_date_place'][0]),re_cate_format['review_date_place'][0])

for t in range(len(re_cate_format['review_date_place'])):
    time_raw = str(re_cate_format['review_date_place'][t]).split('·')[1]
    #若是中文時間格式則取消註解
    '''
    time_raw = time_raw.replace('年','-')
    time_raw = time_raw.replace('月','-')
    time_raw = time_raw.replace('日','')'''
    re_cate_format['review_date_place'][t] = time_raw
re_cate_format['review_date_place'] = pd.to_datetime(re_cate_format['review_date_place'])

print('時間處理後:',type(re_cate_format['review_date_place'][0]),re_cate_format['review_date_place'][0])


時間處理前: <class 'str'> reviewed a product · Mar 21, 2021


/var/folders/9f/cbry14m53b7__fd2jstjs52h0000gn/T/ipykernel_71998/3089670148.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  re_cate_format['review_date_place'][t] = time_raw


時間處理後: <class 'pandas._libs.tslibs.timestamps.Timestamp'> 2021-03-21 00:00:00


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [10]:
re_cate_format.to_excel(modelname+'_Clean_Format.xlsx')

資料後處理 (時間篩選)

In [34]:
start_time = '2018-01-01'
end_time = '2021-12-31'

In [35]:
time_re_cate_format = re_cate_format.loc[(re_cate_format['review_date_place'] >= start_time) #時間起始點
                                       & (re_cate_format['review_date_place'] < end_time)] #時間終點
time_re_cate_format

,user_name,product_name,review_date_place,review_rating,review_content,Asin,Category,Sub_category
0,Jaded Regrets,IMStick.Pro Magnetic Cell Phone Stand - 360 Ro...,2021-11-10,4 out of five stars,I bought this looking for the perfect grip sol...,B09GVVVZ2G,cell phones service plans accessories,cell phone accessories
1,Jaded Regrets,"DisplayPort to HDMI Adapter, DEORNA 4K DP to H...",2021-01-21,3 out of five stars,"I bought this to use for a dual monitor setup,...",B088ZWMRG4,electronics store,computer pc hardware accessories add ons
2,Jaded Regrets,"Mobile Game Controller, Mutop Wireless Gamepad...",2020-02-01,1 out of five stars,I already had a bulky Ipega controller (Ipega-...,B07VB9BWH1,cell phones service plans accessories,cell phone accessories
3,Jaded Regrets,FYSETC 3D Printer A8 A6 Parts Upgrade Heated B...,2019-10-15,5 out of five stars,"This uses a JST-VH 6 pin connector, which is a...",B07JFT3HK8,industrial scientific supplies,Additive Manufacturing Products
4,Jaded Regrets,Hermitshell Travel Case for Oculus Rift + Touc...,2019-07-30,5 out of five stars,"First case I bought for the oculus quest, and ...",B071RWHPWG,electronics store,computer pc hardware accessories add ons
...,...,...,...,...,...,...,...,...
1024,Siddhartha Parepally,"(Renewed) Apple iPhone 8, US Version, 64GB, Sp...",2020-11-17,5 out of five stars,"Great deal, for a perfect phone",B0775MV9K2,cell phones service plans accessories,cell phone devices
1025,Steven,"LED Strip Lights 100ft, RGB 5050 Led Light Str...",2021-10-18,5 out of five stars,"The led light are really good, only thing if y...",B08K4J2X4B,Tools and Home Improvement,Lighting and Ceiling Fans
1026,Lisha Xie,"Upgraded Pet Hair Dryer, 2-in-1 Dog Blow Dryer...",2021-08-24,5 out of five stars,This little brush is compact but with so much ...,B091Z8PTZ7,pet shops dogs cats hamsters kittens,dogs supplies training beds collars grooming
1027,Lisha Xie,"Petaholic Cat Water Fountain, 67oz/2.0L Pet Wa...",2021-08-24,5 out of five stars,This water fountain works quietly and it’s rea...,B09698F3VH,pet shops dogs cats hamsters kittens,cats supplies kittens catnip tree litter


In [36]:
format_by_name = time_re_cate_format.groupby(['user_name'])
name_list = np.unique(time_re_cate_format['user_name'])
print('用戶名單:',name_list[:5],'...')
print('用戶數:',len(name_list))
#format_by_name.get_group(name_list[1])['Sub_category'][:3]

用戶名單: ['A Movie Lover' 'ANOOPREDDY BANAPURAM' 'Aanaan' 'Abdullah A.'
 'Abdullah Qwera'] ...
用戶數: 175


In [37]:


male = 0
female =0
mt = 0
ft =0
for name in name_list:
    m=0
    f=0
    productname = format_by_name.get_group(name)['product_name']
    for pn in productname: #檢測購買男性商品與女性商品用戶數
        if 'Men\'s' in pn:
            m =1
            mt += 1
        elif 'Women\'s' in pn:
            f =1
            ft +=1  
    male += m
    female += f

print('直接購買男性商品的用戶數:',male,'總購買男性商品數',mt,'直接購買女性商品的用戶數:',female,'總購買女性商品',ft)

直接購買男性商品的用戶數: 10 總購買男性商品數 10 直接購買女性商品的用戶數: 6 總購買女性商品 6


關聯式分析(User為購物籃)

Category Association Rule

In [38]:
category_by_name =[]
for name in name_list:
    new_cate = format_by_name.get_group(name)['Category'] #個別用戶的Categorylist
    #print(name,new_cate)
    new_cate = list(new_cate)
    category_by_name.append(new_cate) #For Association Rule
te = TransactionEncoder()
te.ary = te.fit(category_by_name).transform(category_by_name)

In [39]:
Association_df = pd.DataFrame(te.ary, columns=te.columns_)
frequent_items = fpgrowth(Association_df,min_support=0.1,use_colnames=True) #調整support值
frequent_items['length'] = frequent_items['itemsets'].apply(lambda x:len(x))
cate_frequent_format = pd.DataFrame(frequent_items[ (frequent_items['length'] == 1)])
#frequent_format = pd.DataFrame(frequent_items)
cate_frequent_format = cate_frequent_format.sort_values(by = 'support',ascending = False)
cate_frequent_format


,support,itemsets,length
0,0.942857,( electronics store ),1
1,0.320000,( home garden kitchen furniture bedding ),1
3,0.234286,( Tools and Home Improvement ),1
6,0.228571,( cell phones service plans accessories ),1
7,0.211429,( sports outdoors ),1
4,0.188571,( amazon fashion ),1
5,0.142857,( automotive auto truck replacements parts ),1
8,0.120000,( Patio Lawn Garden ),1
2,0.114286,( Beauty Makeup Skin Hair Products ),1


In [41]:
#出現Antecedents條件下出現consequents的機率為confidence
cate_asso_format = association_rules(frequent_items,metric='confidence',min_threshold=0.6) #調整confidence值
cate_asso_format = pd.DataFrame(cate_asso_format[ (cate_asso_format['lift'] > 1)]) #調整lift值
cate_asso_format

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,( cell phones service plans accessories ),( electronics store ),0.228571,0.942857,0.222857,0.975,1.034091,0.007347,2.285714
6,"( cell phones service plans accessories , ...",( electronics store ),0.114286,0.942857,0.108571,0.950,1.007576,0.000816,1.142857


Sub_category Association rule

In [42]:
sub_category_by_name =[]
for name in name_list:
    new_cate = format_by_name.get_group(name)['Sub_category'] #個別用戶的Categorylist
    #print(name,new_cate)
    new_cate = list(new_cate)
    sub_category_by_name.append(new_cate) #For Association Rule
te = TransactionEncoder()
te.ary = te.fit(sub_category_by_name).transform(sub_category_by_name)

In [43]:
sub_Association_df = pd.DataFrame(te.ary, columns=te.columns_)
sub_frequent_items = fpgrowth(sub_Association_df,min_support=0.1,use_colnames=True) #調整support值
sub_frequent_items['length'] = sub_frequent_items['itemsets'].apply(lambda x:len(x))
sub_cate_frequent_format = pd.DataFrame(sub_frequent_items[ (sub_frequent_items['length'] == 1)])
#sub_frequent_format = pd.DataFrame(frequent_items)
sub_cate_frequent_format = sub_cate_frequent_format.sort_values(by = 'support',ascending = False)
sub_cate_frequent_format

,support,itemsets,length
0,0.880000,( Projectors Monitors Computer Add Ons Comput...,1
1,0.234286,( computer pc hardware accessories add ons ),1
5,0.182857,( kitchen dining ),1
2,0.148571,( Camera Photo Film Canon Sony ),1
4,0.142857,( cell phone accessories ),1
3,0.131429,( Accessories Supplies ),1


In [45]:
#出現Antecedents條件下出現consequents的機率為confidence
sub_cate_asso_format = association_rules(sub_frequent_items,metric='confidence',min_threshold=0.6) #調整confidence值
#sub_cate_asso_format = pd.DataFrame(sub_cate_asso_format[ (sub_cate_asso_format['lift'] > 1)]) #調整lift值
sub_cate_asso_format

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,( computer pc hardware accessories add ons ),( Projectors Monitors Computer Add Ons Comput...,0.234286,0.88,0.188571,0.804878,0.914634,-1.760000e-02,0.615
1,( Camera Photo Film Canon Sony ),( Projectors Monitors Computer Add Ons Comput...,0.148571,0.88,0.120000,0.807692,0.917832,-1.074286e-02,0.624
2,( Accessories Supplies ),( Projectors Monitors Computer Add Ons Comput...,0.131429,0.88,0.108571,0.826087,0.938735,-7.085714e-03,0.690
3,( cell phone accessories ),( Projectors Monitors Computer Add Ons Comput...,0.142857,0.88,0.125714,0.880000,1.000000,2.775558e-17,1.000
4,( kitchen dining ),( Projectors Monitors Computer Add Ons Comput...,0.182857,0.88,0.137143,0.750000,0.852273,-2.377143e-02,0.480


In [ ]:
#將特別的itemset進行Breakdown
